In [3]:
import pandas as pd
import os
import numpy as np

In [4]:
df = pd.read_csv("train_y_answer.csv")

In [5]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [6]:
dp = pd.read_csv("public_train_x_dp_full_hashed.csv")

In [7]:
new_dp = dp.copy()
new_dp['NTD_amt'] = new_dp['tx_amt'] * new_dp['exchg_rate']

In [8]:
pub_x = pd.read_csv("public_x_alert_date.csv")

In [9]:
train_x = pd.read_csv("train_x_alert_date.csv")

In [10]:
train_pub_x = pd.concat([pub_x,train_x],axis = 0)

In [11]:
cust_info = pd.read_csv('public_train_x_custinfo_full_hashed.csv')

In [11]:
#cust_info[cust_info['alert_key'].isin(a["alert_key"])].shape

NameError: name 'a' is not defined

In [12]:
df = train_pub_x.merge(cust_info, on = 'alert_key')

In [13]:
cdtx = pd.read_csv("public_train_x_cdtx0001_full_hashed.csv")

In [14]:

cdtx.sort_values("date",inplace=True)
df.sort_values("date",inplace=True)

In [15]:
cdtx_df = pd.merge_asof(cdtx,df,by = "cust_id",on = "date", direction = "forward")

In [16]:
cdtx_df

,cust_id,date,country,cur_type,amt,alert_key,risk_rank,occupation_code,total_asset,AGE
0,f52f83fcc2cf076b66d5a9149b9935021d1d67c99c6112...,0,130,47,109.0,172490.0,3.0,12.0,6416.0,4.0
1,c58e3092e037420bfbde0adaf2f7a4ccb654c5dedac683...,0,130,47,1826.0,171704.0,1.0,9.0,2287681.0,6.0
2,c58e3092e037420bfbde0adaf2f7a4ccb654c5dedac683...,0,130,47,2748.0,171704.0,1.0,9.0,2287681.0,6.0
3,c58e3092e037420bfbde0adaf2f7a4ccb654c5dedac683...,0,130,47,2748.0,171704.0,1.0,9.0,2287681.0,6.0
4,c58e3092e037420bfbde0adaf2f7a4ccb654c5dedac683...,0,130,47,1826.0,171704.0,1.0,9.0,2287681.0,6.0
...,...,...,...,...,...,...,...,...,...,...
1043009,75945c8b1b7af88340ffe4169b39341dfded2c07248d36...,394,134,16,76.0,NaN,NaN,NaN,NaN,NaN
1043010,75945c8b1b7af88340ffe4169b39341dfded2c07248d36...,394,44,0,48.0,NaN,NaN,NaN,NaN,NaN
1043011,75945c8b1b7af88340ffe4169b39341dfded2c07248d36...,394,128,53,1708.0,NaN,NaN,NaN,NaN,NaN
1043012,2886a332d3530dcbfebeb0292251d04c1e0c20e767051d...,394,135,47,1578.0,NaN,NaN,NaN,NaN,NaN


In [17]:
cdtx_df.dropna(subset = ["alert_key"],axis = 0,inplace = True)

In [18]:
cdtx_df["cur_type"].nunique()


43

In [19]:
df = df.merge(cdtx_df.groupby("alert_key").amt.sum().reset_index(),on = "alert_key", how = "left")

In [20]:
df

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE,amt
0,171152,0,7e42b5dca9b28ee8e5545beb834361e90e6197d176b389...,3,13.0,599497.0,6,NaN
1,171584,0,ad09177963d7dffdef4714eb7b25e31c20ae86c046d1e9...,1,17.0,82076.0,3,NaN
2,171616,0,b5682d7daf663b469294340258701dd3febf795648bab8...,3,5.0,167773.0,2,NaN
3,171375,0,4c9914b594bac26a921251f8f7c64f8088c73e57ecb669...,1,9.0,1629378.0,6,NaN
4,171612,0,7de1cdc84de4fe3b420fd370c8d21fa2adaacb35f8c52d...,1,13.0,3121.0,3,NaN
...,...,...,...,...,...,...,...,...
25746,364926,393,f102029c20f1a193dd0a701d9d4b8c1e1fb53f910744ac...,1,17.0,3361306.0,3,2271226.0
25747,364686,393,64ed87c9ddcc4a625b0b39f3e47382e6319c7947dcc518...,3,17.0,2128884.0,3,NaN
25748,364961,393,0af73e104c593d31daa497ea928b04fbf52f2017bd8bb4...,1,12.0,1131590.0,5,NaN
25749,364460,393,fb832de21bafdfae18e043fa51952a719900b74b51924d...,1,13.0,601139.0,5,40651.0


In [21]:
df["wo_amt" ]=False
df. loc[df["amt" ].isna(),"wo_amt"]=True
df = df.fillna(0)
df

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE,amt,wo_amt
0,171152,0,7e42b5dca9b28ee8e5545beb834361e90e6197d176b389...,3,13.0,599497.0,6,0.0,True
1,171584,0,ad09177963d7dffdef4714eb7b25e31c20ae86c046d1e9...,1,17.0,82076.0,3,0.0,True
2,171616,0,b5682d7daf663b469294340258701dd3febf795648bab8...,3,5.0,167773.0,2,0.0,True
3,171375,0,4c9914b594bac26a921251f8f7c64f8088c73e57ecb669...,1,9.0,1629378.0,6,0.0,True
4,171612,0,7de1cdc84de4fe3b420fd370c8d21fa2adaacb35f8c52d...,1,13.0,3121.0,3,0.0,True
...,...,...,...,...,...,...,...,...,...
25746,364926,393,f102029c20f1a193dd0a701d9d4b8c1e1fb53f910744ac...,1,17.0,3361306.0,3,2271226.0,False
25747,364686,393,64ed87c9ddcc4a625b0b39f3e47382e6319c7947dcc518...,3,17.0,2128884.0,3,0.0,True
25748,364961,393,0af73e104c593d31daa497ea928b04fbf52f2017bd8bb4...,1,12.0,1131590.0,5,0.0,True
25749,364460,393,fb832de21bafdfae18e043fa51952a719900b74b51924d...,1,13.0,601139.0,5,40651.0,False


In [ ]:
#df = df.merge(cdtx_df.groupby("alert_key").amt.sum().reset_index(),on = "alert_key", how = "left").fillna(0)

In [22]:
cur_alert = pd.crosstab(cdtx_df["alert_key"],cdtx_df["cur_type"]).reset_index()

In [23]:
df = df.merge(cur_alert,on = "alert_key",how = "left")

In [24]:
df["wo_cur"] = df["wo_amt"]

In [25]:
df = df.fillna(0)

In [ ]:
#df = df.merge(cur_alert,on = "alert_key",how = "left")

In [ ]:
#df = df.merge(cur_alert,on = "alert_key",how = "left").fillna(0)

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE,amt,wo_amt,0,...,43,44,45,46,47,50,53,54,55,56
0,171152,0,7e42b5dca9b28ee8e5545beb834361e90e6197d176b389...,3,13.0,599497.0,6,0.0,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,171584,0,ad09177963d7dffdef4714eb7b25e31c20ae86c046d1e9...,1,17.0,82076.0,3,0.0,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,171616,0,b5682d7daf663b469294340258701dd3febf795648bab8...,3,5.0,167773.0,2,0.0,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,171375,0,4c9914b594bac26a921251f8f7c64f8088c73e57ecb669...,1,9.0,1629378.0,6,0.0,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,171612,0,7de1cdc84de4fe3b420fd370c8d21fa2adaacb35f8c52d...,1,13.0,3121.0,3,0.0,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25743,364729,393,03bbccf0289660b1a481fcfa6741ec764d66152b594889...,1,19.0,0.0,4,0.0,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25745,364699,393,e1bdb85136b523fff6d3cc4721eb604dbd8e500acc3231...,1,12.0,238219.0,2,0.0,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25747,364686,393,64ed87c9ddcc4a625b0b39f3e47382e6319c7947dcc518...,3,17.0,2128884.0,3,0.0,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25748,364961,393,0af73e104c593d31daa497ea928b04fbf52f2017bd8bb4...,1,12.0,1131590.0,5,0.0,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df.head()

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE,amt,wo_amt,0,...,44,45,46,47,50,53,54,55,56,wo_cur
0,171152,0,7e42b5dca9b28ee8e5545beb834361e90e6197d176b389...,3,13.0,599497.0,6,0.0,True,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
1,171584,0,ad09177963d7dffdef4714eb7b25e31c20ae86c046d1e9...,1,17.0,82076.0,3,0.0,True,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
2,171616,0,b5682d7daf663b469294340258701dd3febf795648bab8...,3,5.0,167773.0,2,0.0,True,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
3,171375,0,4c9914b594bac26a921251f8f7c64f8088c73e57ecb669...,1,9.0,1629378.0,6,0.0,True,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
4,171612,0,7de1cdc84de4fe3b420fd370c8d21fa2adaacb35f8c52d...,1,13.0,3121.0,3,0.0,True,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True


In [27]:
remit = pd.read_csv("public_train_x_remit1_full_hashed.csv")

In [28]:
remit.sort_values("trans_date",inplace=True)
df.sort_values("date",inplace=True)

In [29]:
remit

,cust_id,trans_date,trans_no,trade_amount_usd
8237,55466f23ccc268ed9d6487a3115fea61a0f7f59d2222a7...,0,4,4742.0
15151,e7bcfe4ed604a24b9ef86224fdcd01f05ea0a83767f8bf...,0,0,70693.0
7293,46eb67c442f694abfcef7f36fe5a30e7d98addbe6e8d3e...,0,0,7200.0
10565,7e86e893f28634fd266d734e9ec498b237764026a4c8a1...,0,0,5040.0
12763,b5682d7daf663b469294340258701dd3febf795648bab8...,0,0,13370.0
...,...,...,...,...
868,2432456794234739664c8615fa02904e03900ba460b713...,393,0,12343.0
13102,badac61e255c073fc544616596871dcb0346760fec4ae4...,393,0,12202.0
3343,a35ca0ad8421ca39eaf5cb188848f888fb2fe7dc9e1da7...,393,0,138535.0
3865,be7d3c28b1cfcb04e646520dfebdedf1beaf32b89d7d64...,393,0,70833.0


In [30]:
remit_df = pd.merge_asof(remit,df,by = "cust_id",right_on = "date", left_on = "trans_date",direction = "forward")

In [31]:
df = df.merge(remit_df.groupby("alert_key").trade_amount_usd.sum().reset_index(),on = "alert_key",how = "left")

In [32]:
df.head()

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE,amt,wo_amt,0,...,45,46,47,50,53,54,55,56,wo_cur,trade_amount_usd
0,171152,0,7e42b5dca9b28ee8e5545beb834361e90e6197d176b389...,3,13.0,599497.0,6,0.0,True,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,NaN
1,171446,0,95a6c0fd787a7f03473b58897e0d696d4ba27c61003f92...,3,17.0,24379.0,3,879.0,False,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,15524.0
2,171367,0,fa81ef21a193bae90d014d04235a71e7dda88c5bfa6597...,1,12.0,1093612.0,5,3105.0,False,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,False,6187.0
3,171589,0,c782d3ff7148a758fb658540ea5c86845ed2745b03c548...,3,19.0,284269.0,4,0.0,True,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,NaN
4,171760,0,a98c7b12dca3bb6649172c780b2793a701602345ce4b21...,3,19.0,185131.0,6,0.0,True,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,NaN


In [33]:
df["wo_trade_amt" ]=False
df. loc[df["trade_amount_usd" ].isna(),"wo_trade_amt"]=True
df = df.fillna(0)
df

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE,amt,wo_amt,0,...,46,47,50,53,54,55,56,wo_cur,trade_amount_usd,wo_trade_amt
0,171152,0,7e42b5dca9b28ee8e5545beb834361e90e6197d176b389...,3,13.0,599497.0,6,0.0,True,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.0,True
1,171446,0,95a6c0fd787a7f03473b58897e0d696d4ba27c61003f92...,3,17.0,24379.0,3,879.0,False,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,15524.0,False
2,171367,0,fa81ef21a193bae90d014d04235a71e7dda88c5bfa6597...,1,12.0,1093612.0,5,3105.0,False,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,False,6187.0,False
3,171589,0,c782d3ff7148a758fb658540ea5c86845ed2745b03c548...,3,19.0,284269.0,4,0.0,True,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.0,True
4,171760,0,a98c7b12dca3bb6649172c780b2793a701602345ce4b21...,3,19.0,185131.0,6,0.0,True,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25746,364788,393,0ef49d08b4af68e9cb2b2aa63198699ee7dc7b5e1c4550...,2,19.0,28681.0,2,0.0,True,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,78979.0,False
25747,364673,393,f8ce663f765e2459578d285a032a16046fba0b2bba8e12...,1,19.0,213486.0,6,0.0,True,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,1890220.0,False
25748,364626,393,1729de666fbfd3bd42133e8226f5ade5709ef141d8e0b7...,1,19.0,2913.0,2,0.0,True,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.0,True
25749,364999,393,54120a833a4e411f5e15fd3fbdf32e9cd66e000cd6f5ba...,1,18.0,718481.0,3,0.0,True,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,34381.0,False


In [ ]:
#df = df.merge(remit_df.groupby("alert_key").trade_amount_usd.sum().reset_index(),on = "alert_key",how = "left").fillna(0)

In [ ]:
# remit_df.rename(columns = {"0":"remit_0","1":"remit_1","2":"remit_2", "3": "remit_3","4":"remit_4"},inplace = True)

In [34]:
alert_trans_no = pd.crosstab(remit_df["alert_key"],remit_df["trans_no"]).reset_index().rename(columns = {0:"remit_0",1:"remit_1",2:"remit_2", 3: "remit_3",4:"remit_4"})

In [35]:
alert_trans_no

trans_no,alert_key,remit_0,remit_1,remit_2,remit_3,remit_4
0,171367.0,1,0,0,0,0
1,171371.0,0,0,0,0,1
2,171379.0,1,0,0,0,0
3,171446.0,1,0,0,0,0
4,171598.0,1,0,0,0,0
...,...,...,...,...,...,...
5375,364994.0,1,0,0,0,0
5376,364995.0,1,0,0,0,0
5377,364999.0,1,0,0,0,0
5378,365004.0,2,0,0,0,0


In [36]:
df = df.merge(alert_trans_no,on = "alert_key",how = "left")

In [37]:
df["wo_remit" ]=False

df. loc[df[['remit_0','remit_1','remit_2','remit_3','remit_4']].isna().all(1),"wo_remit"]=True

df = df.fillna(0)
df

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE,amt,wo_amt,0,...,56,wo_cur,trade_amount_usd,wo_trade_amt,remit_0,remit_1,remit_2,remit_3,remit_4,wo_remit
0,171152,0,7e42b5dca9b28ee8e5545beb834361e90e6197d176b389...,3,13.0,599497.0,6,0.0,True,0.0,...,0.0,True,0.0,True,0.0,0.0,0.0,0.0,0.0,True
1,171446,0,95a6c0fd787a7f03473b58897e0d696d4ba27c61003f92...,3,17.0,24379.0,3,879.0,False,0.0,...,0.0,False,15524.0,False,1.0,0.0,0.0,0.0,0.0,False
2,171367,0,fa81ef21a193bae90d014d04235a71e7dda88c5bfa6597...,1,12.0,1093612.0,5,3105.0,False,0.0,...,0.0,False,6187.0,False,1.0,0.0,0.0,0.0,0.0,False
3,171589,0,c782d3ff7148a758fb658540ea5c86845ed2745b03c548...,3,19.0,284269.0,4,0.0,True,0.0,...,0.0,True,0.0,True,0.0,0.0,0.0,0.0,0.0,True
4,171760,0,a98c7b12dca3bb6649172c780b2793a701602345ce4b21...,3,19.0,185131.0,6,0.0,True,0.0,...,0.0,True,0.0,True,0.0,0.0,0.0,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25746,364788,393,0ef49d08b4af68e9cb2b2aa63198699ee7dc7b5e1c4550...,2,19.0,28681.0,2,0.0,True,0.0,...,0.0,True,78979.0,False,0.0,0.0,0.0,0.0,2.0,False
25747,364673,393,f8ce663f765e2459578d285a032a16046fba0b2bba8e12...,1,19.0,213486.0,6,0.0,True,0.0,...,0.0,True,1890220.0,False,24.0,0.0,0.0,0.0,0.0,False
25748,364626,393,1729de666fbfd3bd42133e8226f5ade5709ef141d8e0b7...,1,19.0,2913.0,2,0.0,True,0.0,...,0.0,True,0.0,True,0.0,0.0,0.0,0.0,0.0,True
25749,364999,393,54120a833a4e411f5e15fd3fbdf32e9cd66e000cd6f5ba...,1,18.0,718481.0,3,0.0,True,0.0,...,0.0,True,34381.0,False,1.0,0.0,0.0,0.0,0.0,False


In [ ]:
#df = df.merge(alert_trans_no,on = "alert_key",how = "left").fillna(0)

In [38]:
df 

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE,amt,wo_amt,0,...,56,wo_cur,trade_amount_usd,wo_trade_amt,remit_0,remit_1,remit_2,remit_3,remit_4,wo_remit
0,171152,0,7e42b5dca9b28ee8e5545beb834361e90e6197d176b389...,3,13.0,599497.0,6,0.0,True,0.0,...,0.0,True,0.0,True,0.0,0.0,0.0,0.0,0.0,True
1,171446,0,95a6c0fd787a7f03473b58897e0d696d4ba27c61003f92...,3,17.0,24379.0,3,879.0,False,0.0,...,0.0,False,15524.0,False,1.0,0.0,0.0,0.0,0.0,False
2,171367,0,fa81ef21a193bae90d014d04235a71e7dda88c5bfa6597...,1,12.0,1093612.0,5,3105.0,False,0.0,...,0.0,False,6187.0,False,1.0,0.0,0.0,0.0,0.0,False
3,171589,0,c782d3ff7148a758fb658540ea5c86845ed2745b03c548...,3,19.0,284269.0,4,0.0,True,0.0,...,0.0,True,0.0,True,0.0,0.0,0.0,0.0,0.0,True
4,171760,0,a98c7b12dca3bb6649172c780b2793a701602345ce4b21...,3,19.0,185131.0,6,0.0,True,0.0,...,0.0,True,0.0,True,0.0,0.0,0.0,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25746,364788,393,0ef49d08b4af68e9cb2b2aa63198699ee7dc7b5e1c4550...,2,19.0,28681.0,2,0.0,True,0.0,...,0.0,True,78979.0,False,0.0,0.0,0.0,0.0,2.0,False
25747,364673,393,f8ce663f765e2459578d285a032a16046fba0b2bba8e12...,1,19.0,213486.0,6,0.0,True,0.0,...,0.0,True,1890220.0,False,24.0,0.0,0.0,0.0,0.0,False
25748,364626,393,1729de666fbfd3bd42133e8226f5ade5709ef141d8e0b7...,1,19.0,2913.0,2,0.0,True,0.0,...,0.0,True,0.0,True,0.0,0.0,0.0,0.0,0.0,True
25749,364999,393,54120a833a4e411f5e15fd3fbdf32e9cd66e000cd6f5ba...,1,18.0,718481.0,3,0.0,True,0.0,...,0.0,True,34381.0,False,1.0,0.0,0.0,0.0,0.0,False


In [39]:
new_dp["info_asset_code"].unique()

array([16,  6, 13, 20, 12, 15,  5,  1,  2,  4,  3,  0,  8,  9, 14, 17, 10,
       19, 21,  7, 11, 18])

In [40]:
new_dp["cash_deal"] = np.where((new_dp['tx_type']== 1) & (new_dp['info_asset_code'] == 12), True, False)

In [41]:
new_dp.sort_values("tx_date",inplace=True)
df.sort_values("date",inplace=True)

In [42]:
newdp_df = pd.merge_asof(new_dp,df[["alert_key","date","cust_id"]],by = "cust_id",right_on = "date", left_on = "tx_date",direction = "forward")

In [43]:
df = df.merge(newdp_df.groupby(['alert_key'])[["cash_deal","ATM","cross_bank","NTD_amt"]].sum().reset_index(),how = "left", on = "alert_key")

In [44]:
df["wo_deal" ]=False

df. loc[df[["cash_deal","ATM","cross_bank","NTD_amt"]].isna().all(1),"wo_deal"]=True

df = df.fillna(0)
df

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE,amt,wo_amt,0,...,remit_1,remit_2,remit_3,remit_4,wo_remit,cash_deal,ATM,cross_bank,NTD_amt,wo_deal
0,171152,0,7e42b5dca9b28ee8e5545beb834361e90e6197d176b389...,3,13.0,599497.0,6,0.0,True,0.0,...,0.0,0.0,0.0,0.0,True,0.0,1.0,0.0,3.024500e+04,False
1,171209,0,95836236dac30345eaf79b450af364b869724d46adc7cd...,3,12.0,1195038.0,3,226901.0,False,0.0,...,0.0,0.0,0.0,0.0,True,0.0,3.0,0.0,1.568000e+04,False
2,171584,0,ad09177963d7dffdef4714eb7b25e31c20ae86c046d1e9...,1,17.0,82076.0,3,0.0,True,0.0,...,0.0,0.0,0.0,0.0,True,0.0,2.0,0.0,3.032450e+05,False
3,171616,0,b5682d7daf663b469294340258701dd3febf795648bab8...,3,5.0,167773.0,2,0.0,True,0.0,...,0.0,0.0,0.0,0.0,False,0.0,9.0,5.0,2.172000e+03,False
4,171375,0,4c9914b594bac26a921251f8f7c64f8088c73e57ecb669...,1,9.0,1629378.0,6,0.0,True,0.0,...,0.0,0.0,0.0,0.0,True,0.0,3.0,0.0,1.288917e+06,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25746,364926,393,f102029c20f1a193dd0a701d9d4b8c1e1fb53f910744ac...,1,17.0,3361306.0,3,2271226.0,False,0.0,...,0.0,0.0,0.0,0.0,True,0.0,19.0,3.0,5.261339e+06,False
25747,364686,393,64ed87c9ddcc4a625b0b39f3e47382e6319c7947dcc518...,3,17.0,2128884.0,3,0.0,True,0.0,...,0.0,0.0,0.0,0.0,False,0.0,13.0,5.0,5.842548e+05,False
25748,364961,393,0af73e104c593d31daa497ea928b04fbf52f2017bd8bb4...,1,12.0,1131590.0,5,0.0,True,0.0,...,0.0,0.0,0.0,0.0,True,1.0,13.0,8.0,7.983380e+05,False
25749,365008,393,12c9e6d35500d2a96fc2b22a9da8e3deb6048de515a16e...,3,19.0,1230244.0,2,0.0,True,0.0,...,0.0,0.0,0.0,0.0,False,0.0,46.0,41.0,1.370598e+07,False


In [ ]:
#df = df.merge(newdp_df.groupby(['alert_key'])[["cash_deal","ATM","cross_bank","NTD_amt"]].sum().reset_index(),how = "left", on = "alert_key").fillna(0)

In [45]:
df[df["wo_deal"] == True]

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE,amt,wo_amt,0,...,remit_1,remit_2,remit_3,remit_4,wo_remit,cash_deal,ATM,cross_bank,NTD_amt,wo_deal
14,171495,0,deea7839000d9f8c1e3fed53e0a1141266fb8d7149a050...,3,12.0,0.0,3,1007236.0,False,0.0,...,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,0.0,True
15,171606,0,77eb881fb8ad47743816447a9c9f9712f64b1e42bf56fd...,3,12.0,374739.0,3,3090586.0,False,0.0,...,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,0.0,True
21,171612,0,7de1cdc84de4fe3b420fd370c8d21fa2adaacb35f8c52d...,1,13.0,3121.0,3,0.0,True,0.0,...,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,0.0,True
25,171362,0,5664fb8f799d8d846a6396bf118606450244bb7f55f9b6...,1,13.0,462187.0,4,0.0,True,0.0,...,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,0.0,True
29,171369,0,b2ba3c316c162be08338a48cb6e5d9840f2400e41e7362...,1,17.0,13134.0,5,574.0,False,0.0,...,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25708,364675,393,08d2b4ae3c03515cac79354b8f30e3287a8d427f8af2d6...,3,12.0,357.0,3,83276.0,False,0.0,...,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,0.0,True
25728,364770,393,b5c5c235e898032b0dd184993f250ea6bd743c521dc2a5...,1,5.0,715.0,3,52386.0,False,0.0,...,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,0.0,True
25735,364484,393,24110a2ec5a36adb9dbb6540faf32719b82f3b181cc37c...,1,17.0,565453.0,4,45402.0,False,0.0,...,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,0.0,True
25736,364624,393,12da1653493fda99b09d5ca125fbb1d58fa514da8d8f8b...,1,19.0,0.0,5,7767.0,False,0.0,...,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,0.0,True


In [46]:
alert_tx_no = pd.crosstab(newdp_df["alert_key"],newdp_df["tx_type"]).reset_index().rename(columns = {0:"no_0",1:"no_1",2:"no_2", 3: "no_3",4:"no_4"})

In [ ]:
#df = df.merge(alert_tx_no,how = "left",on = "alert_key" ).fillna(0)

In [47]:
df = df.merge(alert_tx_no,how = "left",on = "alert_key" )

In [48]:
df

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE,amt,wo_amt,0,...,remit_4,wo_remit,cash_deal,ATM,cross_bank,NTD_amt,wo_deal,no_1,no_2,no_3
0,171152,0,7e42b5dca9b28ee8e5545beb834361e90e6197d176b389...,3,13.0,599497.0,6,0.0,True,0.0,...,0.0,True,0.0,1.0,0.0,3.024500e+04,False,1.0,1.0,0.0
1,171209,0,95836236dac30345eaf79b450af364b869724d46adc7cd...,3,12.0,1195038.0,3,226901.0,False,0.0,...,0.0,True,0.0,3.0,0.0,1.568000e+04,False,0.0,3.0,0.0
2,171584,0,ad09177963d7dffdef4714eb7b25e31c20ae86c046d1e9...,1,17.0,82076.0,3,0.0,True,0.0,...,0.0,True,0.0,2.0,0.0,3.032450e+05,False,0.0,2.0,0.0
3,171616,0,b5682d7daf663b469294340258701dd3febf795648bab8...,3,5.0,167773.0,2,0.0,True,0.0,...,0.0,False,0.0,9.0,5.0,2.172000e+03,False,0.0,9.0,0.0
4,171375,0,4c9914b594bac26a921251f8f7c64f8088c73e57ecb669...,1,9.0,1629378.0,6,0.0,True,0.0,...,0.0,True,0.0,3.0,0.0,1.288917e+06,False,3.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25746,364926,393,f102029c20f1a193dd0a701d9d4b8c1e1fb53f910744ac...,1,17.0,3361306.0,3,2271226.0,False,0.0,...,0.0,True,0.0,19.0,3.0,5.261339e+06,False,15.0,5.0,0.0
25747,364686,393,64ed87c9ddcc4a625b0b39f3e47382e6319c7947dcc518...,3,17.0,2128884.0,3,0.0,True,0.0,...,0.0,False,0.0,13.0,5.0,5.842548e+05,False,0.0,13.0,0.0
25748,364961,393,0af73e104c593d31daa497ea928b04fbf52f2017bd8bb4...,1,12.0,1131590.0,5,0.0,True,0.0,...,0.0,True,1.0,13.0,8.0,7.983380e+05,False,1.0,28.0,0.0
25749,365008,393,12c9e6d35500d2a96fc2b22a9da8e3deb6048de515a16e...,3,19.0,1230244.0,2,0.0,True,0.0,...,0.0,False,0.0,46.0,41.0,1.370598e+07,False,0.0,46.0,0.0


In [ ]:
df["wo_tx_no" ]=False


KeyError: ignored

In [49]:
df. loc[df[["no_1","no_2","no_3"]].isna().all(1),"wo_tx_no"]=True
df = df.fillna(0)
df

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE,amt,wo_amt,0,...,wo_remit,cash_deal,ATM,cross_bank,NTD_amt,wo_deal,no_1,no_2,no_3,wo_tx_no
0,171152,0,7e42b5dca9b28ee8e5545beb834361e90e6197d176b389...,3,13.0,599497.0,6,0.0,True,0.0,...,True,0.0,1.0,0.0,3.024500e+04,False,1.0,1.0,0.0,0
1,171209,0,95836236dac30345eaf79b450af364b869724d46adc7cd...,3,12.0,1195038.0,3,226901.0,False,0.0,...,True,0.0,3.0,0.0,1.568000e+04,False,0.0,3.0,0.0,0
2,171584,0,ad09177963d7dffdef4714eb7b25e31c20ae86c046d1e9...,1,17.0,82076.0,3,0.0,True,0.0,...,True,0.0,2.0,0.0,3.032450e+05,False,0.0,2.0,0.0,0
3,171616,0,b5682d7daf663b469294340258701dd3febf795648bab8...,3,5.0,167773.0,2,0.0,True,0.0,...,False,0.0,9.0,5.0,2.172000e+03,False,0.0,9.0,0.0,0
4,171375,0,4c9914b594bac26a921251f8f7c64f8088c73e57ecb669...,1,9.0,1629378.0,6,0.0,True,0.0,...,True,0.0,3.0,0.0,1.288917e+06,False,3.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25746,364926,393,f102029c20f1a193dd0a701d9d4b8c1e1fb53f910744ac...,1,17.0,3361306.0,3,2271226.0,False,0.0,...,True,0.0,19.0,3.0,5.261339e+06,False,15.0,5.0,0.0,0
25747,364686,393,64ed87c9ddcc4a625b0b39f3e47382e6319c7947dcc518...,3,17.0,2128884.0,3,0.0,True,0.0,...,False,0.0,13.0,5.0,5.842548e+05,False,0.0,13.0,0.0,0
25748,364961,393,0af73e104c593d31daa497ea928b04fbf52f2017bd8bb4...,1,12.0,1131590.0,5,0.0,True,0.0,...,True,1.0,13.0,8.0,7.983380e+05,False,1.0,28.0,0.0,0
25749,365008,393,12c9e6d35500d2a96fc2b22a9da8e3deb6048de515a16e...,3,19.0,1230244.0,2,0.0,True,0.0,...,False,0.0,46.0,41.0,1.370598e+07,False,0.0,46.0,0.0,0


In [50]:
df[df["wo_tx_no"]==True]

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE,amt,wo_amt,0,...,wo_remit,cash_deal,ATM,cross_bank,NTD_amt,wo_deal,no_1,no_2,no_3,wo_tx_no
14,171495,0,deea7839000d9f8c1e3fed53e0a1141266fb8d7149a050...,3,12.0,0.0,3,1007236.0,False,0.0,...,True,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,True
15,171606,0,77eb881fb8ad47743816447a9c9f9712f64b1e42bf56fd...,3,12.0,374739.0,3,3090586.0,False,0.0,...,True,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,True
21,171612,0,7de1cdc84de4fe3b420fd370c8d21fa2adaacb35f8c52d...,1,13.0,3121.0,3,0.0,True,0.0,...,True,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,True
25,171362,0,5664fb8f799d8d846a6396bf118606450244bb7f55f9b6...,1,13.0,462187.0,4,0.0,True,0.0,...,True,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,True
29,171369,0,b2ba3c316c162be08338a48cb6e5d9840f2400e41e7362...,1,17.0,13134.0,5,574.0,False,0.0,...,True,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25708,364675,393,08d2b4ae3c03515cac79354b8f30e3287a8d427f8af2d6...,3,12.0,357.0,3,83276.0,False,0.0,...,True,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,True
25728,364770,393,b5c5c235e898032b0dd184993f250ea6bd743c521dc2a5...,1,5.0,715.0,3,52386.0,False,0.0,...,True,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,True
25735,364484,393,24110a2ec5a36adb9dbb6540faf32719b82f3b181cc37c...,1,17.0,565453.0,4,45402.0,False,0.0,...,True,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,True
25736,364624,393,12da1653493fda99b09d5ca125fbb1d58fa514da8d8f8b...,1,19.0,0.0,5,7767.0,False,0.0,...,True,0.0,0.0,0.0,0.0,True,0.0,0.0,0.0,True


In [51]:
ccba = pd.read_csv("public_train_x_ccba_full_hashed.csv")

In [52]:
ccba = ccba.drop(["lupay"],axis = 1)

In [53]:
ccba.sort_values("byymm",inplace=True)
df.sort_values("date",inplace=True)

In [54]:
ccba_df = pd.merge_asof(ccba,df[["alert_key","date","cust_id"]],by = "cust_id",right_on = "date", left_on = "byymm",direction = "forward")

In [55]:
ccba_df.columns

Index(['cust_id', 'byymm', 'cycam', 'usgam', 'clamt', 'csamt', 'inamt',
       'cucsm', 'cucah', 'alert_key', 'date'],
      dtype='object')

In [56]:
df = df.merge(ccba_df.groupby("alert_key")[['cycam', 'usgam', 'clamt', 'csamt', 'inamt','cucsm', 'cucah']].sum().reset_index(), how = "left",on = "alert_key")

In [57]:
df["wo_ccba" ]=False
df. loc[df[['cycam', 'usgam', 'clamt', 'csamt', 'inamt','cucsm', 'cucah']].isna().all(1),"wo_ccba"]=True
df = df.fillna(0)
df

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE,amt,wo_amt,0,...,no_3,wo_tx_no,cycam,usgam,clamt,csamt,inamt,cucsm,cucah,wo_ccba
0,171152,0,7e42b5dca9b28ee8e5545beb834361e90e6197d176b389...,3,13.0,599497.0,6,0.0,True,0.0,...,0.0,0,324783.0,64363.0,0.0,0.0,0.0,4981.0,0.0,False
1,171587,0,8a89be5b38f76afa6deeb419c9efd97fca86463faba234...,1,19.0,99703.0,3,0.0,True,0.0,...,0.0,0,104875.0,19475.0,0.0,0.0,0.0,19475.0,0.0,False
2,171446,0,95a6c0fd787a7f03473b58897e0d696d4ba27c61003f92...,3,17.0,24379.0,3,879.0,False,0.0,...,0.0,0,113666.0,97464.0,0.0,0.0,11910.0,16398.0,0.0,False
3,171367,0,fa81ef21a193bae90d014d04235a71e7dda88c5bfa6597...,1,12.0,1093612.0,5,3105.0,False,0.0,...,0.0,0,101816.0,81356.0,0.0,0.0,0.0,78398.0,0.0,False
4,171589,0,c782d3ff7148a758fb658540ea5c86845ed2745b03c548...,3,19.0,284269.0,4,0.0,True,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25746,364788,393,0ef49d08b4af68e9cb2b2aa63198699ee7dc7b5e1c4550...,2,19.0,28681.0,2,0.0,True,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
25747,364673,393,f8ce663f765e2459578d285a032a16046fba0b2bba8e12...,1,19.0,213486.0,6,0.0,True,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
25748,364626,393,1729de666fbfd3bd42133e8226f5ade5709ef141d8e0b7...,1,19.0,2913.0,2,0.0,True,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
25749,364767,393,ff483c74ef2977bfcf04be0a79363af72d1812aac9ce1f...,2,17.0,41066.0,3,0.0,True,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True


In [ ]:
#df = df.merge(ccba_df.groupby("alert_key")[['cycam', 'usgam', 'clamt', 'csamt', 'inamt',
  #     'cucsm', 'cucah']].sum().reset_index(), how = "left",on = "alert_key").fillna(0)

In [58]:
df["occupation_code"].nunique()

21

In [62]:
pip install --upgrade scikit-learn

     |████████████████████████████████| 9.1 MB 341 kB/s eta 0:00:01
     |████████████████████████████████| 297 kB 533 kB/s eta 0:00:01
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2
Note: you may need to restart the kernel to use updated packages.


In [59]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

In [60]:
occu = OneHotEncoder().fit_transform(df[["occupation_code"]])

In [61]:
occu.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [62]:
ohe = OneHotEncoder()
transformed = ohe.fit_transform(df[["occupation_code"]])
df[["occu_"+str(x) for x in ohe.categories_[0]]] = transformed.toarray()

In [63]:
df.drop(["cust_id","occupation_code"],inplace = True, axis = 1)

In [64]:
scaler = MinMaxScaler()
cols = df.columns
df_id = df[["alert_key","date"]]
df_sca = df.drop(["alert_key","date"],axis = 1)

df_sd = scaler.fit_transform(df_sca)
df_sd = pd.DataFrame(df_sd, columns = cols[2:])
df_sd = pd.concat([df_id,df_sd],axis = 1)

TypeError: Feature names are only supported if all input features have string names, but your input has ['int', 'str'] as feature name / column name types. If you want feature names to be stored and validated, you must convert them all to strings, by using X.columns = X.columns.astype(str) for example. Otherwise you can remove feature / column names from your input data, or convert them all to a non-string data type.

In [ ]:
df_sd = df_sd.sort_values("alert_key")

In [ ]:
train_y = pd.read_csv("/content/drive/MyDrive/訓練資料集_first/train_y_answer.csv")

In [ ]:
train_y = train_y.sort_values("alert_key")
train_x = df_sd[df_sd["alert_key"].isin(train_y["alert_key"])]
train_x.shape

(23906, 97)

In [ ]:
a = pd.read_csv("/content/drive/MyDrive/訓練資料集_first/預測的案件名單及提交檔案範例.csv")

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(train_x, train_y['sar_flag'])

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [ ]:
private_x = df_sd[df_sd["alert_key"].isin(a["alert_key"])]
private_x.shape

(1845, 97)

In [ ]:
train_y.shape

(23906, 2)

In [ ]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(train_x, train_y['sar_flag'])
prob_y = clf.predict_proba(private_x)

prob_y = pd.DataFrame(prob_y,columns = ["key","probability"])


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [ ]:
G_NB = GaussianNB()
G_NB.fit(train_x,train_y['sar_flag'])

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


GaussianNB()

In [ ]:
HGB = HistGradientBoostingClassifier()
HGB.fit(train_x,train_y['sar_flag'])
prob_y = HGB.predict_proba(private_x)
prob_y = pd.DataFrame(prob_y,columns = ["key","probability"])

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [ ]:
RF = RandomForestClassifier()
RF.fit(train_x,train_y['sar_flag'])
prob_y = RF.predict_proba(private_x)
prob_y = pd.DataFrame(prob_y,columns = ["key","probability"])

In [ ]:
#smote

HGB = HistGradientBoostingClassifier()
HGB.fit(X_res,y_res)
prob_y = HGB.predict_proba(private_x)
prob_y = pd.DataFrame(prob_y,columns = ["key","probability"])

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [ ]:
prob_y = G_NB.predict_proba(private_x)
prob_y = pd.DataFrame(prob_y,columns = ["key","probability"])

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [ ]:
private_x['alert_key']

23916    352249
23936    352253
23958    352254
23937    352280
23976    352282
          ...  
25651    365001
25652    365004
25722    365008
25653    365009
25667    365073
Name: alert_key, Length: 1845, dtype: int64

In [ ]:
# test = pd.DataFrame([private_x['alert_key'].to_list(),prob_y], columns = ['alert_key','proba'])
# test['proba'] = prob_y
test = pd.concat([private_x["alert_key"].reset_index(),prob_y],axis = 1,)

In [ ]:
ans = pd.merge(a, test[["alert_key","probability"]],how = "left", on = "alert_key")

In [ ]:
ans["probability"] = ans["probability_y"]
ans.loc[ans["probability"].isna(),"probability"] = ans["probability_x"]
ans.drop(["probability_x","probability_y"],axis = 1, inplace = True)

In [ ]:
test

,index,alert_key,key,probability
0,23916,352249,0.999898,0.000102
1,23936,352253,0.999877,0.000123
2,23958,352254,0.999898,0.000102
3,23937,352280,0.999893,0.000107
4,23976,352282,0.999887,0.000113
...,...,...,...,...
1840,25651,365001,0.999961,0.000039
1841,25652,365004,0.999958,0.000042
1842,25722,365008,0.999958,0.000042
1843,25653,365009,0.999960,0.000040


In [ ]:
ans.notna().sum() 

alert_key      3850
probability    3850
dtype: int64

In [ ]:
private_x["alert_key"]

23916    352249
23936    352253
23958    352254
23937    352280
23976    352282
          ...  
25651    365001
25652    365004
25722    365008
25653    365009
25667    365073
Name: alert_key, Length: 1845, dtype: int64

In [ ]:
ans = ans.sort_values("probability",ascending= False)
ans.shape

(3850, 2)

In [ ]:
ans.to_csv("GBT_sm.csv", index = False)

In [ ]:
ans

,alert_key,probability
3849,372119,0.999978
3847,368710,0.999694
3843,371792,0.997107
3840,371595,0.996514
3839,377350,0.996477
...,...,...
3083,353030,0.000120
651,361843,0.000117
3234,357850,0.000110
523,358593,0.000095
